In [706]:
import pandas as pd
import os
import random
import numpy as np
# import math
import csv


def get_files(root_path, nr_files):
    if nr_files < 1 or nr_files > 2:
        print(f"Number of files {nr_files} is not 1 or 2")
        return None
        
    sampled = []
    file_paths = []
    
    # Loop over all files in the root path folder and store full path
    for file in os.listdir(root_path):
        folder_path = os.path.join(root_path, file)
        
        # Search for folders in the full path and if found, store all files in lists
        if os.path.isdir(folder_path):
            files_within_folders = os.listdir(folder_path)
            
            # Filter out only csv files
            csv = [i for i in files_within_folders if i.endswith('.csv')]
            
            # Getting full path of the non-empty csv file
            for each in csv:
                file_path = (os.path.join(root_path, folder_path, each))
                if os.path.getsize(file_path) > 0:
                    file_paths.append(file_path)
                else:
                    print(f"File {file_path} is empty")
                    return None
            
            # Sample based on the input provided
            if nr_files < len(csv):
                sampled.extend(random.sample(csv, nr_files))
                # print(f"2: {sampled}")
            else:
                sampled.extend(csv)
                # print(f"1: {sampled}")


    dataf = []
    for i in sampled:
        for j in file_paths:
            if j.endswith(i):
                dataf.append(j)
                break

    return(dataf)
    

dfs = get_files("C:\\Users\\dimak\\Documents\\stock_price_data_files", 2)


print(dfs)

['C:\\Users\\dimak\\Documents\\stock_price_data_files\\LSE\\FLTR.csv', 'C:\\Users\\dimak\\Documents\\stock_price_data_files\\LSE\\GSK.csv', 'C:\\Users\\dimak\\Documents\\stock_price_data_files\\NASDAQ\\TSLA.csv', 'C:\\Users\\dimak\\Documents\\stock_price_data_files\\NYSE\\ASH.csv', 'C:\\Users\\dimak\\Documents\\stock_price_data_files\\NYSE\\NMR.csv']


In [708]:
for i in dfs:
    df = pd.read_csv(i)
    # print(df)

    std_dev_threshold = 2
    
    def finding_consecutive_rand_val(df):
        random_nr = random.randint(0, len(df)-29)
        array_30_random_val = df.iloc[random_nr-2:random_nr+28, 2]
        # print(array_30_random_val)
        return array_30_random_val

    result = finding_consecutive_rand_val(df)
    # print(result)

    
    def find_outliers(result):
        mean_price = np.mean(result)
        # avg_price = sum(result) / len(result)
        standard_dev = np.std(result)
        # std_dev = math.sqrt((len(result) * sum(x**2 for x in result) - (sum(result))**2) / (len(result) * (len(result) - 1)))

        # Obtaining using the z-score method, the outliers
        if standard_dev != 0:
            z_score = [(elem - mean_price) / standard_dev for elem in result]
            outliers = [i for i in z_score if (abs(i) > std_dev_threshold)]
            # return "Mean ", mean_price, "Outliers", outliers
            return mean_price, outliers
        else:
            return 0
            print ("For current array, the standard deviation is 0")
 
        
    outlier = find_outliers(result)
    # print("find_outliers func = ", outlier)

    
    def export_csv(df, outlier):
        ticker_value = df.iloc[0, 0]
        timestamp = df.iloc[0, 1]
        price_value = df.iloc[0, 2]

        # print("out0 = ", outlier[0], "out1 = ", outlier[1])

        # Assign headers for each value if outliers
        if outlier[1]:
            for item in outlier[1]:
                columns = {
                        "Ticker": [ticker_value],
                        "Timestamp": [timestamp],
                        "Price value": [price_value],
                        "Mean price": [outlier[0]],
                        "Price-Mean": [price_value - outlier[0]],
                        "Deviation %": [item - std_dev_threshold],
                        "Outliers": [item]
                        }
                df = pd.DataFrame(columns)

                # Creating new name for the output file
                file_name = os.path.basename(i)
                new_file_name = f"new_{file_name}"

                # Append all the results containing outliers
                with open(new_file_name, 'a', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerow(df.columns)
                    for index, row in df.iterrows():
                        writer.writerow(row)
                
                print(f"CSVs saved as: {new_file_name}")

    outcome = export_csv(df, outlier)


# cwd = os.getcwd()
# print(cwd)

CSVs saved as: new_GSK.csv
CSVs saved as: new_TSLA.csv
CSVs saved as: new_ASH.csv
